## CONCILIACIÓN RETENCIONES MX

**ENTRADAS**

1. Base de datos Liliana 

2. Movimientos MX (

*query
)*

3. Ajustes manuales MX (

*query*

4. Compensaciones MX (

query)

**PROCESO**

1. Encontrar ajustes automáticos y manuales hechos al valor de la retención de cada RT en la base de Liliana 

2. Encontrar compensaciones hechas al valor de la retención de cada RT en la base de Liliana 

3. Hallar la diferencia total entre RET IVA en base de Liliana en relación a los ajustes y compensaciones 

4. Hallar la diferencia total entre RET ISR en base de Liliana en relación a los ajustes y compensaciones

5. Hallar la diferencia total entre TOTAL RET en base de Liliana en relación a los ajustes y compensaciones


**SALIDAS** 

1. Archivo de excel que contenga las diferencias entre los valores descritos en el proceso 

## 1. Cargar bases de datos 

**Librerias**

In [1]:
#Importar librerias 
import pandas as pd 
import numpy as np
import snowflake
import sqlalchemy
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine

**CREDENCIALES SNOWFLAKE**

In [2]:
#Credenciales no ingresadas en repositorio publico por clausula de privacidad
snowflake_credentials = {'url' : ,
'account' : ,
'user':,
"authenticator": "externalbrowser",  
#"password":"0lXVwz8hV*vEA!",                            
'warehouse': ,
'database' : 
}

In [3]:
engine = sqlalchemy.create_engine(URL(**snowflake_credentials), encoding = 'latin-1')
connection = engine.connect()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


**DEFINICIÓN DE LAS FECHAS**

In [4]:
fecha_inicio = input('Fecha de inicio: ')
fecha_final = input('Fecha de fin: ')

Fecha de inicio: 2021-09-01
Fecha de fin: 2021-09-30


**QUERY DE MOVIMIENTOS**

In [5]:
#PRUEBA
warehouse = pd.read_sql_query("USE NOMBRE_WAREHOUSE",connection)
movimientos = pd.read_sql_query(
f'''query''', connection)

In [7]:
movimientos_agrupados = pd.pivot_table(movimientos, values='valor_movimientos', index=['storekeeper_id'],
                    columns=['ajuste'], aggfunc=np.sum, fill_value=0)

In [8]:
movimientos_agrupados

ajuste,isr_automatico,isr_manual_movimientos,isr_penalty,iva_automatico,iva_manual_movimientos,iva_penalty,state_tax
storekeeper_id,,,,,,,
12,-8.59,0.00,0.00,-21.88,-0.01,0.00,0.0
64,-27.87,-0.31,0.00,-67.00,0.00,0.00,0.0
99,-405.07,-20.91,0.00,-997.62,-55.70,0.00,0.0
138,-23.90,0.00,0.00,-91.03,-0.03,0.00,0.0
139,-189.52,-30.00,0.00,-386.53,-104.14,0.00,0.0
...,...,...,...,...,...,...,...
656230,-1.53,0.00,-13.04,-2.63,0.00,-2.63,0.0
656233,-2.53,0.00,-21.55,-6.44,0.00,-6.44,0.0
656245,-1.49,0.00,-12.63,-3.64,0.00,-3.64,0.0


In [9]:
movimientos_agrupados.to_excel('movimientos_agrupados.xlsx')

**BASE DE LILIANA**

In [10]:
mes_bd = input("Número mes revision:")

Número mes revision:09


In [11]:
bd_liliana = pd.read_csv(f'bd_liliana_{mes_bd}.csv', sep = ';')

In [12]:
bd_liliana.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65659 entries, 0 to 65658
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   storekeeper_id  65659 non-null  int64  
 1   concat          65659 non-null  object 
 2   nombre          65659 non-null  object 
 3   Penalizado      65659 non-null  object 
 4   Facturado       65659 non-null  object 
 5   RET_ISR         65659 non-null  float64
 6   RET_IVA         65659 non-null  float64
 7   Error           10244 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 4.0+ MB


**AJUSTES MANUALES** 

In [13]:
mes = input('Mes de los ajustes(primera en mayuscula): ')

Mes de los ajustes(primera en mayuscula): Septiembre


In [14]:
fecha_inicio_ajustes = input('Fecha inicio ajustes: ')
fecha_final_ajustes = input('Fecha final ajustes: ')

Fecha inicio ajustes: 2021-09-01
Fecha final ajustes: 2021-09-30


In [15]:
warehouse = pd.read_sql_query("USE WAREHOUSE PAGOS_ANALYSTS",connection)
ajustes_manuales = pd.read_sql_query(
f'''query
''', connection)

In [16]:
ajustes_agrupados = pd.pivot_table(ajustes_manuales, values='valor_ajustes_manuales', index=['storekeeper_id'],
                    columns=['tipo_ajuste'], aggfunc=np.sum, fill_value=0)

In [18]:
ajustes_agrupados.to_excel('ajustes_agrupados.xlsx')

**COMPENSACIONES**

In [19]:
mes_inicio_compensaciones = input("Mes inicio compensaciones: ")
mes_final_compensaciones = input("Mes final compensaciones: ")

Mes inicio compensaciones: 2021-09-01
Mes final compensaciones: 2021-09-30


In [20]:
warehouse = pd.read_sql_query("USE WAREHOUSE PAGOS_ANALYSTS",connection)
compensaciones = pd.read_sql_query(
f'''query''', connection)


In [22]:
compensaciones_agrupadas = pd.pivot_table(compensaciones, values='valor_compensacion', index=['storekeeper_id'],
                    columns=['tipo_compensacion'], aggfunc=np.sum, fill_value=0)

In [23]:
compensaciones_agrupadas.to_excel('compensaciones_agrupadas.xlsx')

## 2. Realizar cruces de la bases de datos

In [26]:
bd_liliana.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65659 entries, 0 to 65658
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   storekeeper_id  65659 non-null  int64  
 1   concat          65659 non-null  object 
 2   nombre          65659 non-null  object 
 3   Penalizado      65659 non-null  object 
 4   Facturado       65659 non-null  object 
 5   RET_ISR         65659 non-null  float64
 6   RET_IVA         65659 non-null  float64
 7   Error           10244 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 4.0+ MB


In [27]:
bdliliana_movimientos = bd_liliana.merge(movimientos_agrupados,how = 'left',left_on='storekeeper_id',right_on='storekeeper_id')

In [30]:
bdliliana_movimientos_ajustes = bdliliana_movimientos.merge(ajustes_agrupados,how='left', left_on='storekeeper_id', right_on='storekeeper_id')

In [32]:
tabla_final = bdliliana_movimientos_ajustes.merge(compensaciones_agrupadas,how='left',left_on='storekeeper_id',right_on='storekeeper_id') 

In [33]:
tabla_final.fillna(0,inplace=True)

## 3. Organizar valores de los campos 

In [35]:
tabla_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65659 entries, 0 to 65658
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   storekeeper_id          65659 non-null  int64  
 1   concat                  65659 non-null  object 
 2   nombre                  65659 non-null  object 
 3   Penalizado              65659 non-null  object 
 4   Facturado               65659 non-null  object 
 5   RET_ISR                 65659 non-null  float64
 6   RET_IVA                 65659 non-null  float64
 7   Error                   65659 non-null  object 
 8   isr_automatico          65659 non-null  float64
 9   isr_manual_movimientos  65659 non-null  float64
 10  isr_penalty             65659 non-null  float64
 11  iva_automatico          65659 non-null  float64
 12  iva_manual_movimientos  65659 non-null  float64
 13  iva_penalty             65659 non-null  float64
 14  state_tax               65659 non-null

In [36]:
try:
    for i in tabla_final.index:
        tabla_final.loc[i,'diferencia_iva_query_ajustes'] = tabla_final.loc[i,'RET_IVA'] + tabla_final.loc[i,'iva_compensacion'] + tabla_final.loc[i,'iva_automatico'] + tabla_final.loc[i,'iva_ajustes_manual'] + tabla_final.loc[i,'iva_penalty'] + tabla_final.loc[i,'state_tax'] 
        tabla_final.loc[i,'diferencia_isr_query_ajustes'] = tabla_final.loc[i,'RET_ISR'] + tabla_final.loc[i,'isr_compensacion'] + tabla_final.loc[i,'isr_automatico'] + tabla_final.loc[i,'isr_ajustes_manual'] + tabla_final.loc[i,'isr_penalty']   
        
except:
    for i in tabla_final.index:
        tabla_final.loc[i,'diferencia_iva_query_ajustes'] = tabla_final.loc[i,'RET_IVA'] + tabla_final.loc[i,'iva_compensacion'] + tabla_final.loc[i,'iva_automatico'] + tabla_final.loc[i,'iva_ajustes_manual']  
        tabla_final.loc[i,'diferencia_isr_query_ajustes'] = tabla_final.loc[i,'RET_ISR'] + tabla_final.loc[i,'isr_compensacion'] + tabla_final.loc[i,'isr_automatico'] + tabla_final.loc[i,'isr_ajustes_manual']  
    


In [37]:
tabla_final = tabla_final.round(2)

In [38]:
tabla_final.to_excel(f'C:/ruta/conciliaciones/conciliacion_{mes}.xlsx')

In [39]:
try:
    
    for i in tabla_final.index:
        tabla_final.loc[i,'diferencia_iva'] = tabla_final.loc[i,'RET_IVA'] + tabla_final.loc[i,'iva_compensacion'] + tabla_final.loc[i,'iva_automatico'] + tabla_final.loc[i,'iva_manual_movimientos'] + tabla_final.loc[i,'iva_penalty'] + tabla_final.loc[i,'state_tax'] 
        tabla_final.loc[i,'diferencia_isr'] = tabla_final.loc[i,'RET_ISR'] +  tabla_final.loc[i,'isr_compensacion'] + tabla_final.loc[i,'isr_automatico'] + tabla_final.loc[i,'isr_manual_movimientos'] + tabla_final.loc[i,'isr_penalty']   

except:
    for i in tabla_final.index:
        tabla_final.loc[i,'diferencia_iva'] = tabla_final.loc[i,'RET_IVA'] + tabla_final.loc[i,'iva_compensacion'] + tabla_final.loc[i,'iva_automatico'] + tabla_final.loc[i,'iva_manual_movimientos'] 
        tabla_final.loc[i,'diferencia_isr'] = tabla_final.loc[i,'RET_ISR'] +  tabla_final.loc[i,'isr_compensacion'] + tabla_final.loc[i,'isr_automatico'] + tabla_final.loc[i,'isr_manual_movimientos']   
    